In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

sys.path.append("helpers")
from helpers import LORIS_data_preparation

In [2]:
# LORIS saved query (all data)
full = pd.read_csv("../../diagnosis_predictor/data/raw/LORIS-release-10.csv", dtype=object)

In [3]:
data = LORIS_data_preparation.prepare_data(full)

DEBUG helper


/Users/home/Repos/hbn/HBN-scripts/exploration/helpers/LORIS_data_preparation.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full["ID"] = full_for_EID_check.iloc[:, 0]


In [12]:
wisc_vci = "WISC,WISC_VCI"
wisc_bd = "WISC,WISC_BD_Scaled"
wisc_mr = "WISC,WISC_MR_Scaled"
wisc_vsi = "WISC,WISC_VSI"
wisc_vci = "WISC,WISC_VCI"
wisc_fri = "WISC,WISC_FRI"
wasi_bd = "WASI,WASI_BD_T"
wasi_mr = "WASI,WASI_Matrix_T"
wasi_vci = "WASI,WASI_VCI_Comp"
wasi_pri = "WASI,WASI_PRI_Comp"
wais_bd = "WAIS,WAIS_BD_PERC"
wais_mr = "WAIS,WAIS_MR_PERC"
wais_vci = "WAIS,WAIS_VCI_COMP"
wais_pri = "WAIS,WAIS_PRI_COMP"
assq = "ASSQ,ASSQ_Total"
word = "WIAT,WIAT_Word_P"
cbcl = "CBCL,CBCL_SP_T"
cbcl_pre = "CBCL_Pre,CBCLPre_SP_T"
num = "WIAT,WIAT_Num_P"
flanker_p = "NIH_Scores,NIH7_Flanker_P"
card_p = "NIH_Scores,NIH7_Card_P"
flanker = "NIH_final,NIH_Flanker_Age_Corr_Stnd"
card = "NIH_final,NIH_Card_Age_Corr_Stnd"
peg = "Pegboard,peg_z_d"

cols = [wisc_vci, 
        wisc_bd, 
        wisc_mr,
        wisc_vsi,
        wisc_vci,
        wisc_fri,
        wasi_bd,
        wasi_mr,
        wasi_vci,
        wasi_pri,
        #wais_bd,
        #wais_mr,
        #wais_vci,
        #wais_pri,
        assq,
        word,
        cbcl,
        cbcl_pre,
        num,
        flanker,
        card,
        flanker_p,
        card_p,
        peg]

for col in cols:
    if col in data.columns:
        data[col] = data[col].astype(float)

data = data[data[assq] < 19]
    
# Step 1
spatial_deficit_wisc = (data[wisc_bd] <= 7) | (data[wisc_mr] <= 7)
spatial_deficit_wasi = (data[wasi_bd] <= 40) | (data[wasi_mr] <= 40)
#spatial_deficit_wais = (data[wais_bd] <= 16 | data[wais_mr] <= 16)

spatial_deficit = (spatial_deficit_wisc | spatial_deficit_wasi) #| spatial_deficit_wais)

discrepancy_wisc_vci_fri =((data[wisc_vci] - data[wisc_fri]) > 15)
discrepancy_wisc_vci_vsi = ((data[wisc_vci] - data[wisc_vsi]) > 15)
discrepancy_wasi = ((data[wasi_vci] - data[wasi_pri]) > 15)
#discrepancy_wais = (data[wais_vci] - data[wais_pri]) > 15

discrepancy = discrepancy_wisc_vci_fri | discrepancy_wisc_vci_vsi | discrepancy_wasi #| discrepancy_wais

spatial_condition = (spatial_deficit | discrepancy)
reading_condition = (data[word] >= 16)
step_1_condition = (spatial_condition & reading_condition)

# Step 2
nih_p = (data[flanker_p] < 16) | (data[card_p] < 16)
#nih = (data[flanker] <= 85) | (data[card] <= 85)

EF_condition = (nih_p) #| (nih)

social_condition = (data[cbcl] >= 70) | (data[cbcl_pre] >= 70)
math_condition = (data[num] <= 16)
motor_condition = (data[peg] <= -0.800)
step2_condition = ((social_condition.astype(int) + math_condition.astype(int) + EF_condition.astype(int) + motor_condition.astype(int)) >= 2)

# Step 3
asd_condition = (data[assq] < 19)
step_3_condition = asd_condition

In [24]:
print("Only VCI-VSI > 15")
print(spatial_condition.value_counts()[True] / spatial_condition.value_counts().sum())
print("VCI-VSI > 15 and WIAT_Word_P > 16")
print((spatial_condition & reading_condition).value_counts()[True] / (spatial_condition & reading_condition).value_counts().sum())
print("VCI-VSI > 15 and EF (Flanker or Card) < 16")
print((spatial_condition & EF_condition).value_counts()[True] / (spatial_condition & EF_condition).value_counts().sum())
print("VCI-VSI > 15 and WIAT_Num_P <= 16")
print((spatial_condition & math_condition).value_counts()[True] / (spatial_condition & math_condition).value_counts().sum())
print("VCI-VSI > 15 and CBCL SP T > 70")
print((spatial_condition & social_condition).value_counts()[True] / (spatial_condition & social_condition).value_counts().sum())




Only VCI-VSI > 15
0.15243608297153882
VCI-VSI > 15 and WIAT_Word_P > 16
0.12662807525325614
VCI-VSI > 15 and EF (Flanker or Card) < 16
0.08007718282682104
VCI-VSI > 15 and WIAT_Num_P <= 16
0.02653159671972986
VCI-VSI > 15 and CBCL SP T > 70
0.015195369030390739


In [14]:
data["NVLD"] = step_1_condition & step2_condition & step_3_condition
data["NVLD"].value_counts()

False    2936
True      299
Name: NVLD, dtype: int64

In [15]:
# Correlation between NVLD and other PreInt_DevHx,skill_age_06
data["PreInt_DevHx,skill_age_06"] = data["PreInt_DevHx,skill_age_06"].astype(float)

data["PreInt_DevHx,skill_age_06"].corr(data["NVLD"])


0.045775547920479595

In [28]:
# Print pegboard scores for NVLD and non-NVLD
print(data[data["NVLD"] == True]["Pegboard,peg_z_d"].mean())
print(data[data["NVLD"] == False]["Pegboard,peg_z_d"].mean())

# Print data[wisc_vci] - data[wisc_fri] discrepancy means for NVLD and non-NVLD where columns are present
data["FRI-VCI"] = data[wisc_vci] - data[wisc_fri]
print(data[data["NVLD"] == True]["FRI-VCI"].mean(axis=0))
print(data[data["NVLD"] == False]["FRI-VCI"].mean(axis=0))

# Correlation between discrepancy and PreInt_DevHx,skill_age_06
print(data["FRI-VCI"].corr(data["PreInt_DevHx,skill_age_06"]))

# Correlation between MR and PreInt_DevHx,skill_age_06 and between BD and PreInt_DevHx,skill_age_06
print(data[wisc_mr].corr(data["PreInt_DevHx,skill_age_06"]))
print(data[wisc_bd].corr(data["PreInt_DevHx,skill_age_06"]))

-1.528145579124579
-0.07261860379384395
8.302583025830259
3.1625766871165646
0.039606243980566114
-0.07403985713887366
-0.06389175044353447
